## Project Discussion

###### Question 1: Summarize for us the goal of this project and how machine learning is useful in trying to accomplish it. As part of your answer, give some background on the dataset and how it can be used to answer the project question. Were there any outliers in the data when you got it, and how did you handle those?  [relevant rubric items: “data exploration”, “outlier investigation”]

The dataset investigated as part of this project includes the Enron dataset. The dataset included 10 income variables, 4 variables regarding each individual stock options, and finally five more variables that included summary information on the number of emails sent and received from/to persons of interest.

The goal of this project was to develop a classification model that can be useful in identfying the persons of interest in the dataset from known features including income and email summary information.

The dataset included approximately 150 records. Of these, there were two clear outliers. The first resulted from a "conversion" error. The "TOTAL" line was rolled into the dataset as a separate entity in addition to all the other records in the dataset. The other bad record involved an entity "THE TRAVEL AGENCY IN THE PARK" which could have been a business entity providing travel/concierge services for Enron employees and hence should have excluded from the dataset. In addition to these two records, I noticed that all the features corresponding to "LOCKHART EUGENE E" contained NaN and hence did not provide any useful information in the context of this study. Therefore, I truncated these three records from the overall dataset. In addition to these aforementioned outliers/bad data, I noticed that the person-level data for "BELFER ROBERT" and "BHATNAGAR SANJAY" were slightly off from what was provided in the pdf document included in the project folder. Assuming the pdf represents the correct information, I set the field values to reflect the same information as provided in the pdf document.

The modeling approach involved using "poi" as the target variable with all other features as independent/explanatory variables. While each classification algorithm attempts to solve this problem slightly differently, the general approach involves identifying the most significant features that can explain the variability in the response variable. In case of this project, the target variable was a binary variable and the income and email summary features were the independent variables.

###### Question 2: What features did you end up using in your POI identifier, and what selection process did you use to pick them? Did you have to do any scaling? Why or why not? As part of the assignment, you should attempt to engineer your own feature that does not come ready-made in the dataset -- explain what feature you tried to make, and the rationale behind it. (You do not necessarily have to use it in the final analysis, only engineer and test it.) In your feature selection step, if you used an algorithm like a decision tree, please also give the feature importances of the features that you use, and if you used an automated feature selection function like SelectKBest, please report the feature scores and reasons for your choice of parameter values.  [relevant rubric items: “create new features”, “properly scale features”, “intelligently select feature”]

Once the outliers are accounted for, I decided to drop two of the features entirely from the dataset (total_payments and total_stock_value) as they were sum totals of the income and stock option features for each of the records in the dataset. Therefore, these features provided no additional information beyond what was already available from the remianing income variables.

Secondly, I realized that several of the income features were collinear. For example, high annual bonuses were often associated with high salaries, thus making them highly correlated. A summary of the correlation coefficients are given in Table 1.

###### Table 1: Pairwise Correlation Coefficients of Income Variables

| | salary | exercised_stock_options | bonus | restricted_stock | expenses | loan_advances | deferred_income | long_term_incentive | other |
| --- |:---:| ---:| --- |:---:| ---:| --- |:---:| ---:| --- |
| salary | 1.000 | 0.616 | 0.742 | 0.559 | 0.345 | 0.420 | -0.367 | 0.557 | 0.591 |
| exercised_stock_options | 0.616 | 1.000 | 0.639 | 0.750 | 0.106 | 0.725 | -0.220 | 0.480 | 0.720 |
| bonus | 0.742 | 0.639 | 1.000 | 0.539 | 0.250 | 0.563 | -0.340 | 0.459 | 0.520 |
| restricted_stock | 0.559 | 0.750 | 0.539 | 1.000 | 0.186 | 0.606 | -0.134 | 0.349 | 0.656 |
| expenses | 0.345 | 0.106 | 0.250 | 0.186 | 1.000 | 0.130 | -0.070 | 0.076 | 0.151 |
| loan_advances | 0.420 | 0.725 | 0.563 | 0.606 | 0.130 | 1.000 | -0.035 | 0.435 | 0.770 |
| deferred_income | -0.367 | -0.220 | -0.340 | -0.134 | -0.070 | -0.035 | 1.000 | -0.279 | -0.358 |
| long_term_incentive | 0.557 | 0.480 | 0.459 | 0.349 | 0.076 | 0.435 | -0.279 | 1.000 | 0.587 |
| other | 0.591 | 0.720 | 0.520 | 0.656 | 0.151 | 0.770 | -0.358 | 0.587 | 1.000

Therefore, any one variable can be used as a representative for the entire cluster as all the other variables that are correlated with the first provide the same information. Hence, I tried to reduce the number of dimensions. I retained the top two principal components for the income features which together captured 96% (eigenvalue_1 = 0.86, eigenvalue_2 = 0.096) of the variability in the income variables.

In the next step, I transformed the number of emails sent by a person to a poi and those received by a person from a poi as ratios with respect to the total number of messages sent and received, respectively. This was primarily done based on the intuition that the number of emails sent or received from poi-s will be naturally higher for those who sent or received more emails from the list of people in their address book. However, computing a ratio will standardize the data and remove aberrations arising because of differences in scale.

It should be noted that the synthetic features were developed with the objective of reducing the number of dimensions and therefore the feature selection process as part of the logistic regression model is actually part of the logistic regression model development. The statistical significance  of the negineered features results from the logistic regression model were not back-propagated to develop the features.

Following this, I restricted my dataset to five independent variables including the two transformed income variables, the ratios of emails sent and received from poi-s and lastly the number of emails shared with poi-s. Lastly, 
I standardized (mean = 0, sd = 1) the dataset to make sure each of the features are in the same scale to avoid dominance of one feature over the other. While the last step was not necessary for logistic regression or tree-based classifiers, support vector classifiers are sensitive to scales of magnitude (1).

###### Question 3: What algorithm did you end up using? What other one(s) did you try? How did model performance differ between algorithms?  [relevant rubric item: “pick an algorithm”]

My final classification algorithm was decision tree with tree depth of 2 and gini-index as the information criterion. While I tried a variety of classification algorithms, I chose to retain decision trees as my final classification algorithm due to higher precision, recall, and F1 score. It should be noted that in coming up with the final specification for the decision tree classifier, I chose to retain features that showed a relatively high information score (feature importance) as those with lower information scores add noise to the predictions. To that effect, my final decision tree classifier only uses the count of emails shared between a person and a poi and the ratio of emails received from pois w.r.t. the count of all emails received by that person. A brief summary of the significant features used for the final iteration of each classifier is provided below in Table 2.

###### Table 2: Final Feature Set used for each Algorithm and Significance

| Classification Algorithm        | Significant Features  | Statistical Significance/Gini-index |
| ------------- |:-------------|:--------|
| Decision Trees      |  shared_with_poi; from_poi_to_person_ratio | gini_index: 0.72, 0.28 |
| Naive Bayes      |  finance_1; finance_2; shared_with_poi; from_poi_to_person_ratio; from_person_to_poi | NA |
| Random Forest |  finance_1; shared_with_poi; from_poi_to_person_ratio | gini_index: 0.38, 0.35, 0.26 |
| Logistic Regression |  finance_1; finance_2; shared_with_poi | p_value: 0.44, 0.37, 0.01 |

In addition to the decision tree classifier, I also tried logistic regression, naive bayes classifier, random forest, and support vector classifier. During development of the logit model, I initially tried with all five features previously described but I dropped some in favor of others to minimize the number of features that had relatively poor p-values. I also considered including interaction effects but owing to their low statistical significance values, I chose to retain the specification that offered the highest accuracy and included all the statistically significant features.

In case of the random forest model, I approached the problem in a similar fashion but I used the gini-index to narrow down to the list of significant features.

With the Naive Bayes classifier, I retained all five variables as there wasn't a way to differentiate between the significant and not-so significant features.

While the summary information for each of these classifiers is tabulated above, there are a few points that are worth mentioning in this context:
1. In case of the logistic regression classifier, the initial test results indicated the highest precision (0.67) and recall (0.4) values among all the other classifiers. However, when evaluating the same model using the stratified samples, the recall dropped below 0.3.
2. I evaluated the support vector classifier for the Enron data but unfortunately it failed to identify even one of the poi-s correctly thus ending up with zero for both precision and recall. Therefore, I dropped the support vector classifier from further consideration.
3. Lastly, the variable, emails_shared_with_poi proved to be the most significant explanatory variable for identifying the poi-s. This was demonstrated by its highest statistical significance in the logistic regression setting as well as the highest information score with the tree-based classifiers.

###### Question 4: What does it mean to tune the parameters of an algorithm, and what can happen if you don’t do this well?  How did you tune the parameters of your particular algorithm? (Some algorithms do not have parameters that you need to tune -- if this is the case for the one you picked, identify and briefly explain how you would have done it for the model that was not your final choice or a different model that does utilize parameter tuning, e.g. a decision tree classifier).  [relevant rubric item: “tune the algorithm”]

The tuning of an algorithm refers to optimizing the hyperparameters of the model so that the predictions obtained from the classifier align with the measured labels. The lack of tuning can mean a world of difference as the predictions obtained will be significantly different from the observed values, thus resulting in exceptionally high prediction errors.

For the logistic regression specification, I focused on two separate parameters - complexity_parameter (12 levels) and class_weight (2 levels). I used the gridSearchCV method avaialble in the sklearn library to determine the most optimal combination of these hyperparameters to develop my final logistic regression model.

With the decision tree classifier, I chose to optimize five separate hyperparameters including the information criterion, depth of the tree, class_weight, minimum number of samples required at a node for further splitting, and minimum number of samples for a leaf node. The optimal hyperparameter values for minimum number of samples for splitting a node and the minimum samples at a leaf node were 2 and 5 (relatively high number of samples for each leaf node), respectively. Moreover the maximum depth of the tree was limited to 2 levels (shallow tree). In summary, the hyperparameter values when put together suggests that efforts were made to prevent overfitting of the classifier to the training dataset.

While the gridSearchCV offers robust optimization capabilities for a variety of machine-learning algorithms, one needs to use caution when coming up with the factorial for the hyperparameters. For example, while optimizing the hyperparameters for the random forest model, I initially included seven levels for the number of estimators ranging from 100 to 10000. This resulted in exceptionally long run-times as there were five other hyperparameters that were also being optimized simulataneously. Eventually, I dropped 2 levels and ended up with 5 levels ranging between 100 and 2000. The important conclusion to draw from this exercise was that while marginal improvements may be possible by fine adjustments of the hyperparameters, it is equally important to consider the performance implications of these improvements.

###### Question 5: What is validation, and what’s a classic mistake you can make if you do it wrong? How did you validate your analysis?  [relevant rubric item: “validation strategy”]

Validation is the process of evaluating the trained model on a dataset that was not used to train the model. A classic mistake with the validation process will be using the training dataset to validate the model in which case the model will prove to be extremely accurate as the model may have memorized the training data. However, when used for testing, the same model will fail to deliver the accuracy previously witnessed as the model might have been overfitted to the training data. In this project, I used the train_test_split function from the sklearn library to split the data and held 30% of the overall data for validation.

Another important consideration for this study was the unbalanced nature of the dataset as the number of non-poi-s dominated over the poi-s. Hence, it was necessary to make sure that the training and evaluation datasets were split such that the proportion of poi-s in each set resembled that is the full dataset, else we run the risk of biasing the model one way or the other. The stratified shuffle split used in this project accounts for this.

###### Question 6: Give at least 2 evaluation metrics and your average performance for each of them.  Explain an interpretation of your metrics that says something human-understandable about your algorithm’s performance. [relevant rubric item: “usage of evaluation metrics”]

The evaluation metrics provided in Table 3 correspond to the most optimal choice of features for each of the algorithms used in this study. The final evaluation for each algorithm was performed using the helper function provided in tester.py (test_classifier) which uses "stratified shuffle split". The stratified shuffle split returns an user-defined number of random splits wherein the ratio of samples belonging to each class is maintained as the original dataset.

###### Table 3: Evaluation Metrics computed using the Optimal Feature Choice for each Algorithm

| Classification Algorithm        | Precision           | Recall  | F1 score  |
| ------------- |:-------------:| -----:|
| Decision Trees      | 0.42 |  0.55 |  0.48 |
| Naive Bayes      | 0.44      |    0.32 |  0.37 |
| Random Forest | 0.35      |     0.40 |  0.37 |
| Logistic Regression | 0.57      |     0.20 |  0.30 |

My final classifier of choice was decision tree which produced a precision and recall value of 0.42 and 0.55, respectively. However, according to the project's framework, the reviewer requested that a comparison be made among the algorithms used while using the identical set of features across the board. Given the decision tree classifier's highest precision, recall, and F1 score, I chose to use the same two features (number of emails shared with poi and the ratio of emails received by the person from a poi to the total number of emails received) as I used with the decision tree. Table 4 summarizes the results from this exercise. While we see the precision, recall, and F1 scores dropping by using the new set of features for logistic regression and naive bayes classifier, the same resulted in a significant improvement for the random forest model. This could be because of the identical measures of significance used (gini-index) for both decision tree and random forest classifiers

###### Table 4: Evaluation Metrics computed using Identical Features (Feature Set used for DTree)

| Classification Algorithm        | Precision           | Recall  | F1 score  |
| ------------- |:-------------:| -----:|
| Decision Trees      | 0.42 |  0.55 |  0.48 |
| Naive Bayes      | 0.20      |    0.11 |  0.14 |
| Random Forest | 0.46      |     0.52 |  0.49 |
| Logistic Regression | 0.19      |     0.02 |  0.03 |

Recall refers to the proportion of correctly identified records out of all the relevant records (in this case, the poi-s). Precision refers to the proportion of correctly identified records out of all identified records (i.e., both true positives and false positives). So in the context of the decision tree classifier, it can be summarized that the classifier is able to identify more than 1 out of every 2 poi-s in the dataset. On the contrary, approximately 2 out of every 5 poi-s identified by the decision tree classifer are actually poi-s.

#### References

1. http://stats.stackexchange.com/questions/65094/why-scaling-is-important-for-the-linear-svm-classification Accessed on 09/12/2016.